In [ ]:
import os
import numpy as np
import pandas as pd
from keras.applications import ResNet50
from keras.applications.vgg19 import preprocess_input
from keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.python.keras.models import load_model
from tensorflow.python.layers.core import Dropout

In [ ]:
image_size = (224, 224)
image_size_ext = (224, 224, 3)
batch_size = 5
sample_data = "../../sample-data"
CLASSES = ["Cyclone", "Earthquake", "Flood", "Wildfire"]


In [ ]:
def get_random_images(count: int):
    images = []
    labels = []

    train_data_dir = "../data/"
    for sub_dir in os.listdir(train_data_dir):
        image_list = os.listdir(
            os.path.join(train_data_dir, sub_dir)
        )  #list of all image names in the directory
        image_list = list(map(lambda x: os.path.join(sub_dir, x), image_list))
        images.extend(image_list)
        labels.extend([sub_dir] * len(image_list))

    df = pd.DataFrame({"Images": images, "Labels": labels})
    df = df.sample(frac=1).reset_index(drop=True)  # To shuffle the data
    df = df.head(count)  # to take the subset of data (I'm taking 100 from it)

    aug = ImageDataGenerator(
        rotation_range=30,
        zoom_range=0.15,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.15,
        horizontal_flip=True,
        validation_split=0.2,
    )

    train_data = aug.flow_from_dataframe(
        dataframe=df,
        directory=train_data_dir,
        x_col="Images",
        y_col="Labels",
        batch_size=32,
        seed=42,
        shuffle=True,
        class_mode="categorical",
        target_size=image_size,
        subset="training"
    )

    validation_data = aug.flow_from_dataframe(
        dataframe=df,
        directory=train_data_dir,
        x_col="Images",
        y_col="Labels",
        batch_size=32,
        seed=42,
        shuffle=True,
        class_mode="categorical",
        target_size=image_size,
        subset="validation"
    )
    return train_data, validation_data



In [ ]:
#ensure the layers are not trained. In order words, the weights are used as is
baseModel = ResNet50(
    weights="imagenet", include_top=False,
    input_shape=(224, 224, 3)
)

# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(CLASSES), activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
    layer.trainable = False

#check the model architecture
model.summary()

In [ ]:
#compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')

In [ ]:
train_data, validation_data = get_random_images(10)

In [ ]:
history = model.fit_generator(train_data, validation_data=validation_data, epochs=2)


In [ ]:
folder_path = '../Test/'
res = []
for img in os.listdir(folder_path):
    img = os.path.join(folder_path, img)
    print(img)
    my_image = load_img(img, target_size=(224, 224))

    #preprocess the image
    my_image = img_to_array(my_image)
    my_image = my_image.reshape((1, my_image.shape[0], my_image.shape[1], my_image.shape[2]))
    my_image = preprocess_input(my_image)

    #make the prediction
    prediction = model.predict(my_image)
    for x in prediction:
        res.append([np.round(x), img])


In [ ]:
for each in res:
    print(each)

# 0-> Cyclone
#     Eathquake
#     Floods
#     WildFires

In [ ]:
model.save("../model.h5")

In [ ]:
model = load_model('../model.h5')
model.summary()
